Based on the Prototype 2 (AD-PMM) algorithm of Shefi and Teboulle 2014, for convex QPs of the form:
$$
\begin{align}
\text{mimimise } &\frac{1}{2} x^T P x + c^T x \\
\text{subject to } & Ax + s = b \\
& s \in \mathcal{K}
\end{align}
$$
where $P \in \mathbb{S}^{n}_+$, $c \in \mathbb{R}^n$, $A \in \mathbb{R}^{m \times n}$, $b \in \mathbb{R}^m$, and $\mathcal{K} = \mathcal{K}_1 \times \ldots \times \mathcal{K}_l$, with $l \leq m$ and where each cone in the Cartesian product is either a nonnegative orthant or a zero cone.

For a generic square symmetric matrix $P$, we use the notation/partition
$$
P = D(P) + R(P),
$$
where $D(P)$ is diagonal with the same entries as those in the main diagonal of $P$. Thus $R(P)$ is the remainder—entries not in the diagonal of $P$.

In [2]:
# Get the required libraries
using LinearAlgebra, Printf
using SparseArrays
using JuMP
using Clarabel, ClarabelBenchmarks

In [8]:
# Define the QP problem data
struct QPProblem
    P::AbstractMatrix{Float64}  # PSD matrix in R^{n x n}
    c::Vector{Float64}  # Vector in R^n
    A::AbstractMatrix{Float64}  # Matrix in R^{m x n}
    b::Vector{Float64}  # Vector in R^m
    M1::AbstractMatrix{Float64}
    K::Vector{Clarabel.SupportedCone}
    τ::Float64
    ρ::Float64
    
    
    # Constructor with a check for P being PSD
    function QPProblem(P::AbstractMatrix{Float64},
        c::Vector{Float64},
        A::AbstractMatrix{Float64},
        b::Vector{Float64},
        M1::AbstractMatrix{Float64},
        K::Vector{Clarabel.SupportedCone},
        τ::Float64,
        ρ::Float64)
        # Check if P is symmetric (special handling for sparse matrices)
        if !(issymmetric(P) || (P isa SparseMatrixCSC && isequal(P, P')))
            error("Matrix P must be symmetric.")
        end
        
        # Check positive semidefiniteness of P
        if minimum(eigvals(Matrix(P))) < 0
            error("Matrix P must be positive semidefinite (PSD).")
        end

        # Check positive semidefiniteness of M1
        n = size(P, 1)
        if minimum(eigvals(Matrix(M1))) < 0
            error("Matrix M1 must be PSD. Current smallest eigval: $(minimum(eigvals(Matrix(M1))))")
        end

        # Check if K (cones) is a valid vector of Clarabel.SupportedCone
        if !(K isa Vector{Clarabel.SupportedCone})
            error("K must be a vector of Clarabel.SupportedCone.")
        end
        
        # Create the instance of the struct
        new(P, c, A, b, M1, K, τ, ρ) # what to store in struct, to be accessed later
    end
end

In [9]:
# Auxiliary function: Extracts the diagonal of a square matrix (works with sparse/dense matrices)
function diag_part(A::AbstractMatrix{Float64})
    if A isa SparseMatrixCSC
        return spdiagm(0 => diag(A))  # Create a sparse diagonal matrix
    else
        return Diagonal(diag(A))     # Create a dense diagonal matrix
    end
end

# Auxiliary function: Returns a matrix with zero diagonal and other entries the same as A
function off_diag_part(A::AbstractMatrix{Float64})
    if A isa SparseMatrixCSC
        return A - spdiagm(0 => diag(A))  # Subtract the diagonal in sparse form
    else
        return A - Diagonal(diag(A))      # Subtract the diagonal in dense form
    end
end

off_diag_part (generic function with 1 method)

In [10]:
# Select problem and load data
# Outer (first) dictionary is the problem class.   Inner (second) is the problem name. 
all_problems = ClarabelBenchmarks.PROBLEMS
problem = all_problems["maros"]["HS21"]

# This disables problem data-scaling.   You could enable it to get a better conditioned problem,
# but then any convergence checks would be checking something different compared to other solvers
optimizer = optimizer_with_attributes(Clarabel.Optimizer,"equilibrate_enable"=>false)

# create and populate a solver, but don’t solve
model = problem(Model(optimizer); solve = false) 

# extract the Clarabel solver object from the JuMP `model` wrapper 
solver = model.moi_backend.optimizer.model.optimizer.solver

# extract the problem data 
P = solver.data.P
A = solver.data.A
(m, n) = size(A)
c = solver.data.q
b = solver.data.b
K = solver.data.cones # K is a list of cones, each cone is a dictionary

# NB: here, you could check that K is a vector of length one, containing a single 
# entry of type Clarabel.NonnegativeConeT.
if length(K) == 1 && typeof(K[1]) == Clarabel.NonnegativeConeT
    println("Note: only a single, nonnegative cone! Simples")
end

Note: only a single, nonnegative cone! Simples


In [22]:
# # Initialise QP data
# n = 3  # number of variables
# m = 3  # number of (inequality) constraints

# P = [5.0 0 1; 0 6 6; 1 6 8]
# c = [1.0, 2, 3]
# b = -1 * ones(Float64, m)
# A = Matrix{Float64}(I, n, n)

# Step size parameters chosen by the user
ρ = 1.0 # (dual)

# Choose algorithm based on M1 construction
# int in {1, 2, 3, 4}
variant = 4

if variant == 1 # NOTE: most "economical"
    take_away = off_diag_part(P + ρ * A' * A)
elseif variant == 2 # NOTE: least "economical"
    take_away = P + ρ * A' * A
elseif variant == 3 # NOTE: intermediate
    take_away = P + off_diag_part(ρ * A' * A)
elseif variant == 4 # NOTE: also intermediate
    take_away = off_diag_part(P) + ρ * A' * A
else
    error("Invalid variant.")
end

# Choose primal step size as 90% of maximum allowable while to keep M1 PSD
τ = 1 / maximum(eigvals(Matrix(take_away)))
M1 = (1 / τ) * Matrix{Float64}(I, n, n) - take_away
println("Dual step size ρ: $ρ")
println("Primal step size τ: $τ")

Dual step size ρ: 1.0
Primal step size τ: 0.009708737864077669


# Solve with our method

In [23]:
#TODO: implement residual calculation
# termination criteria; look at COSMO paper.

# Can later try against SCS or COSMO

# mul! function for multiplication? or better to just count iterations for now (code is more readable)
# benchmarks from Clarabel paper --- need to extract problem data, maybe tricky

# Create a problem instance
problem = QPProblem(P, c, A, b, M1, K, τ, ρ)

# Initialize sequences x, s, and y
x = ones(n)
s = ones(m)
y = ones(m)

# Define the iteration function
function iterate!(problem::QPProblem, x::Vector{Float64}, s::Vector{Float64}, y::Vector{Float64})
    # Unpack problem data for easier reference
    P, c, A, b, M1, K, τ, ρ = problem.P, problem.c, problem.A, problem.b, problem.M1, problem.K, problem.τ, problem.ρ

    
    ### Step 1: Update x ###
    # Kept in general form here; may or may not be a diagonal system, depending on choice of M1
    x_new = x - (M1 + P + ρ * A' * A) \ (P * x + c + A' * (y + ρ * (A * x + s - b)))

    ### Step 2: Update s ###
    # At this moment allows for cone K = K_1 \times \ldots \K_l,
    # with each cone in the product either nonnegative orthant or zero cone.
    s_new = zeros(Float64, size(A, 1))
    start_idx = 1
    for cone in K
        end_idx = start_idx + cone.dim - 1
        s_slice = b[start_idx:end_idx] - A[start_idx:end_idx, :] * x_new - y[start_idx:end_idx] / ρ

        # Project portion of s depending on the cone type
        if cone isa Clarabel.NonnegativeConeT
            s_new[start_idx:end_idx] = max.(s_slice, 0)
        elseif cone isa Clarabel.ZeroConeT
            s_new[start_idx:end_idx] = zeros(Float64, cone.dim)
        else
            error("Unsupported cone type: $typeof(cone)")
        end
        
        start_idx = end_idx + 1
    end

    s_new = max.(b - A * x_new - y / ρ, 0)

    ### Step 3: Update y ###
    y_new = y + ρ * (A * x_new + s_new - b)

    # Update the sequences
    x .= x_new
    s .= s_new
    y .= y_new
end

# Main iteration loop
max_iters = 500  # Maximum number of iterations (adjust as needed)
for k in 0:max_iters
    # Print each iteration result with formatted output on a single line
    if k % 50 == 0
        @printf("Iteration %4.1d: x = [%s]  s = [%s]  y = [%s]\n",
            k,
            join(map(xi -> @sprintf("%12.5e", xi), x), ", "),
            join(map(zi -> @sprintf("%12.5e", zi), s), ", "),
            join(map(yi -> @sprintf("%12.5e", yi), y), ", "))
    end
    
    iterate!(problem, x, s, y)
end

# Print final solution on a single line
println("\nPrototype FOM results:")
@printf("x = [%s]  s = [%s]  y = [%s]\n",
    join(map(xi -> @sprintf("%12.5e", xi), x), ", "),
    join(map(zi -> @sprintf("%12.5e", zi), s), ", "),
    join(map(yi -> @sprintf("%12.5e", yi), y), ", "))

Iteration    0: x = [ 1.00000e+00,  1.00000e+00]  s = [ 1.00000e+00,  1.00000e+00,  1.00000e+00,  1.00000e+00,  1.00000e+00]  y = [ 1.00000e+00,  1.00000e+00,  1.00000e+00,  1.00000e+00,  1.00000e+00]
Iteration   50: x = [ 2.27778e+00,  3.67519e-01]  s = [ 4.77222e+01,  4.96325e+01,  1.24102e+01,  2.77776e-01,  5.03675e+01]  y = [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00]
Iteration  100: x = [ 2.25577e+00,  1.40500e-01]  s = [ 4.77442e+01,  4.98595e+01,  1.24172e+01,  2.55771e-01,  5.01405e+01]  y = [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00]
Iteration  150: x = [ 2.23398e+00,  5.37121e-02]  s = [ 4.77660e+01,  4.99463e+01,  1.22861e+01,  2.33978e-01,  5.00537e+01]  y = [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00]
Iteration  200: x = [ 2.21240e+00,  2.05337e-02]  s = [ 4.77876e+01,  4.99795e+01,  1.21034e+01,  2.12396e-01,  5.00205e+01]  y = [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e

# Solve with OSQP/SCS

In [ ]:
using OSQP

# Convert problem data to sparse type
P_osqp = sparse(P)
A_osqp = sparse(A)
# Create OSQP object
prob = OSQP.Model()
# Set up workspace and change alpha parameter
OSQP.setup!(prob; P=P_osqp, q=c, A=A_osqp, u=b, alpha=1, verbose=false)
# Solve problem
results = OSQP.solve!(prob);

# Print OSQP results
println("OSQP results:")
println("x = [", join(map(v -> @sprintf("%12.5e", v), results.x), ", "), "]")
println("y = [", join(map(v -> @sprintf("%12.5e", v), results.y), ", "), "]")

OSQP results:
x = [-1.00000e+00, -1.00000e+00, -1.00000e+00]
y = [ 5.00000e+00,  1.00000e+01,  1.20000e+01]
